In [ ]:
!pip install transformers

# Imports

In [3]:
import requests
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForTokenClassification, TFBertForTokenClassification

# Dataset Processing

In [109]:
datas = pd.read_csv('/content/drive/MyDrive/Gene entities/500_test_documents.csv', sep='|', header=None).rename(columns={0:'Doc', 1:'Type', 2:'Content'})


In [ ]:
data = datas.iloc[:21, :]
datas

# Pipeline

In [80]:
PRETRAINED = "pruas/BENT-PubMedBERT-NER-Gene"
ner = pipeline(task="ner",model=PRETRAINED, tokenizer=PRETRAINED)


In [72]:
ner(txt, aggregation_strategy='first')

[{'entity_group': 'B',
  'score': 0.9824869,
  'word': 'alpha',
  'start': 171,
  'end': 176},
 {'entity_group': 'I',
  'score': 0.9499634,
  'word': '( 2b ) - adrenergic receptor',
  'start': 176,
  'end': 200},
 {'entity_group': 'B',
  'score': 0.95005727,
  'word': 'adra2b',
  'start': 207,
  'end': 213},
 {'entity_group': 'B',
  'score': 0.96451855,
  'word': 'adra2b',
  'start': 770,
  'end': 776}]

In [129]:
data['Entities'] = data.content.apply(lambda x: {ent['entity_group']: ent['word'] for ent in ner(x, aggregation_strategy='first')})

<ipython-input-129-fe05e22a03ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Entities'] = data.content.apply(lambda x: {ent['entity_group']: ent['word'] for ent in ner(x, aggregation_strategy='first')})


In [ ]:
ner(datas.content.iloc[21], aggregation_strategy='first')

In [25]:
def gene_extract(prediction):

    words = []
    prev_end = 0
    for id, sets in enumerate(prediction):
        start, end = sets['start'], sets['end']     # Token index
        if id == 0:
            words.append(txt[start: end])
            prev_end = end
        elif start == prev_end:
            words.append(txt[start: end])
            prev_end = end
        elif start != prev_end:
            words.append(' ')
            words.append(txt[start: end])
            if (start - prev_end) >= 1:
              words.append(',')
              #print(txt[start: end])
            prev_end = end
    return ("").join(words)

In [26]:
gene_extract(result)

'12-nucleotide insertion,/deletion (,indel) polymorph, promoter, region, alpha,(2B)-adrenergic receptor, gene, ADRA,2B Gen,omic DNA, ind,el AD,RA2B ind,el polymorph, position, -,48 start, cod,on far, upstream, region, promoter,'